In [1]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import random
from mnist1d.data import make_dataset, get_dataset_args
from training_scripts.class_incremental_model import ClassIncrementalModel





In [2]:
def training_loop_verification(input_size, num_classes, class_wise_data, class_dataloaders_test, save_path, lr = 0.001, num_samples = 32):
    # define the model and the parameters
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = ClassIncrementalModel(input_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # dataset parameter
    batch_size = 32
    max_cycles = 5000

    # define the training loop
    for cycle in range(max_cycles):

        # create dataset for each class
        class_datasets = {}
        for i in range(num_classes):
            sampled_data = random.sample(class_wise_data[i],
                                         min(len(class_wise_data[i]), num_samples))  # Permute and limit to num_samples
            class_datasets[i] = TensorDataset(torch.stack([torch.FloatTensor(item[0]) for item in sampled_data]),
                                              torch.LongTensor([item[1] for item in sampled_data]))

        # create a dataloader for each class
        class_dataloaders = {}
        for i in range(num_classes):
            class_dataloaders[i] = DataLoader(class_datasets[i], batch_size=batch_size, shuffle=True)

        for cla in range(num_classes):
            model.train()
            for inputs, labels in class_dataloaders[cla]:
                # Training step
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        torch.cuda.empty_cache()
        del inputs, labels, outputs
    return model

In [3]:

defaults = get_dataset_args()

defaults.num_samples = 40000

data = make_dataset(defaults)
x, y, t = data['x'], data['y'], data['t']
x_test, y_test = data['x_test'], data['y_test']

print(x.shape, y.shape)
print(x_test.shape, y_test.shape)

input_size = x.shape[1]
num_classes = len(set(y))
print("Input size", input_size)
print("Number of classes", num_classes)

# testing data

class_wise_data_test = [[] for _ in range(num_classes)]  # Create 10 separate lists

for i in range(len(y)):
    clas = int(y[i])
    class_wise_data_test[clas].append((x[i], clas))

# create dataset for each class
class_datasets_test = {}
for i in range(10):
    sampled_data = class_wise_data_test[i]
    class_datasets_test[i] = TensorDataset(torch.stack([torch.FloatTensor(item[0]) for item in sampled_data]),
                                           torch.LongTensor([item[1] for item in sampled_data]))

# create a dataloader for each class
class_dataloaders_test = {}
for i in range(10):
    class_dataloaders_test[i] = DataLoader(class_datasets_test[i], batch_size=32, shuffle=True)

class_wise_data = [[] for _ in range(num_classes)]  # Create 10 separate lists

for i in range(len(y)):
    clas = int(y[i])
    class_wise_data[clas].append((x[i], clas))

num_classes_iter = [10]
learning_rate_iter = [1e-3]
num_samples_iter = [1024]


for num_samples in num_samples_iter:
    for num_classes in num_classes_iter:
        for lr in learning_rate_iter:
            print("Starting experiment with the following parameters")
            print(num_samples, num_classes, lr)
            save_path = "results" + str(num_samples) + "_" + str(num_classes) + "_" + str(lr) + ".csv"
            model = training_loop_verification(input_size, num_classes, class_wise_data, class_dataloaders_test, save_path, lr = lr, num_samples = num_samples)

(32000, 40) (32000,)
(8000, 40) (8000,)
Input size 40
Number of classes 10
Starting experiment with the following parameters
1024 10 0.001


In [4]:
# Test the model on the entire test set
model.eval()
# First create a TensorDataset from your test data
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))

# Then create the DataLoader from the dataset
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Accuracy of the model on the test images: 86 %


In [6]:
# SAve the model weights
torch.save(model.state_dict(), "ten_class_best_cyclic_weights.pt")